<a href="https://colab.research.google.com/github/seonae0223/Deep_Learning/blob/main/05_Model_Save_%EB%B0%8F_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FashionMNIST Dataset & DataLoader 구성

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# FashionMNIST 데이터세트 다운로드
train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26421880/26421880 [00:02<00:00, 9393610.90it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 177155.55it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3281706.98it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6137656.91it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader  = DataLoader(test_data, batch_size=64, shuffle=False)

# 모델링

In [ ]:
from torch import nn

class NeuralNetwork(nn.Module):

  def __init__(self):
    # nn.Module 생성
    super(NeuralNetwork, self).__init__()

    # 레이어 정의

    # 평탄화 레이어 정의
    self.flatten = nn.Flatten()

    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 128),
        nn.ReLU(),
        nn.Linear(128, 10)
    )

  def forward(self, x):
    # 입력되는 x의 모양이 (64, 1, 28, 28) -> (N, C, H, W)
    #    입력 이미지에 대한 평탄화가 필요
    x = self.flatten(x) # flatten 레이어를 지나게 되면 (64, 784)
    y = self.linear_relu_stack(x)

    return y

# 모델 생성

In [ ]:
import torch

# cuda 환경(GPU)을 사용할 수 있으면 device='cuda', 아니면 device='cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


# 모델 훈련

In [ ]:
loss_fn = nn.CrossEntropyLoss() # 이미 여기에 소프트맥스 함수가 포함되어 있다.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# 훈련 과정
#  데이터 로딩 -> 예측 -> Loss 계산 -> 미분(backward) -> 최적화
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)

  # 모델을 훈련 모드로 설정
  model.train()

  for batch, (X, y) in enumerate(dataloader):
    # 데이터 로더에 들어있던 텐서들을 모델과 같은 위치(모델이 gpu니까 데이터도 gpu)로 옮기기
    X, y = X.to(device), y.to(device)

    # 예측 (forward)
    pred = model(X) # softmax가 적용 안 되어있음

    # Loss 계산
    loss = loss_fn(pred, y) # loss function 내에서 softmax가 적용 된 다음 y에 대한 loss를 구한다.

    # 역전파 수행
    optimizer.zero_grad() # 이전 배치에 남아있는 기울기를 제거
    loss.backward() # 오차 역전파
    optimizer.step()

    # 배치가 100번 돌 때마다 화면에 출력
    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"Train Loss : {loss:>7f} [ {current:>5d} / {size:>5d} ]")

In [ ]:
def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)

  # loss는 배치 별로 계산, correct는 전체 데이터 세트에 대한 평균 정확도
  test_loss, correct = 0, 0

  # 모델을 추론 모드로 바꿔준다.
  model.eval()

  # 추론 과정에서는 기울기를 구할 필요가 없음. 따라서 모든 파라미터(model.parameters())의 required_grad=False
  with torch.no_grad():
    for X, y in dataloader:

      X, y = X.to(device), y.to(device)

      pred = model(X)

      # test_loss를 배치마다 구해서 더해주기
      test_loss += loss_fn(pred, y).item()

      # 10개의 예측 값중 가장 큰 곳의 인덱스를 argmax로 찾고, 타겟(y)와 일치하는지 확인
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  # 배치 개수 구하기
  num_batches = len(dataloader)

  # Loss 평균 구하기
  test_loss /= num_batches

  # Accuracy 구하기
  correct /= size

  print(f"Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg Loss : {test_loss:>8f}\n")

In [ ]:
epochs = 10

for i in range(epochs):
  print(f"Epoch {i+1}\n......................")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)

print("훈련 끝!")

Epoch 1
......................
Train Loss : 2.306376 [     0 / 60000 ]
Train Loss : 1.540602 [  6400 / 60000 ]
Train Loss : 1.090355 [ 12800 / 60000 ]
Train Loss : 0.785369 [ 19200 / 60000 ]
Train Loss : 0.873697 [ 25600 / 60000 ]
Train Loss : 0.758562 [ 32000 / 60000 ]
Train Loss : 0.569963 [ 38400 / 60000 ]
Train Loss : 0.506294 [ 44800 / 60000 ]
Train Loss : 0.585362 [ 51200 / 60000 ]
Train Loss : 0.508171 [ 57600 / 60000 ]
Test Error : 
 Accuracy : 79.1%, Avg Loss : 0.627077

Epoch 2
......................
Train Loss : 0.514188 [     0 / 60000 ]
Train Loss : 0.548960 [  6400 / 60000 ]
Train Loss : 0.454546 [ 12800 / 60000 ]
Train Loss : 0.836860 [ 19200 / 60000 ]
Train Loss : 0.494870 [ 25600 / 60000 ]
Train Loss : 0.690731 [ 32000 / 60000 ]
Train Loss : 0.490407 [ 38400 / 60000 ]
Train Loss : 0.566538 [ 44800 / 60000 ]
Train Loss : 0.491988 [ 51200 / 60000 ]
Train Loss : 0.581397 [ 57600 / 60000 ]
Test Error : 
 Accuracy : 81.1%, Avg Loss : 0.540995

Epoch 3
......................

# 훈련된 모델의 가중치를 저장 / 불러오기
불러올 곳에서 **모델의 구조를 알고 있는 경우** 가중치만 저장하면 적은 용량으로 저장하고 불러오는 것이 가능합니다.

In [ ]:
# model.state_dict() : 모델 내에 있는 레이어 별 가중치를 들고 있는 딕셔너리
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[ 0.0262, -0.0189,  0.0225,  ..., -0.0284,  0.0393,  0.0326],
                      [-0.0095,  0.0039, -0.0591,  ...,  0.0091,  0.0519, -0.0246],
                      [-0.0296, -0.0125,  0.0474,  ...,  0.0777,  0.0672,  0.0175],
                      ...,
                      [ 0.0062, -0.0273,  0.0348,  ...,  0.0011,  0.0257, -0.0310],
                      [ 0.0298, -0.0471, -0.0626,  ..., -0.0160,  0.0197, -0.0248],
                      [-0.0135, -0.0419, -0.0089,  ...,  0.0316,  0.0686,  0.0367]],
                     device='cuda:0')),
             ('linear_relu_stack.0.bias',
              tensor([-0.0770,  0.1002, -0.0528, -0.1268,  0.0101, -0.0249,  0.2492,  0.0348,
                      -0.1589, -0.0118, -0.0146,  0.3299,  0.1381,  0.1135,  0.1075,  0.2467,
                       0.0408,  0.1602,  0.0586,  0.0515, -0.0794,  0.0252,  0.3029,  0.0481,
                       0.0150,  0.0771, -0.0020,  0.0716, -0

In [ ]:
# 저장된 가중치 파일(pth) 불러오기
model2 = NeuralNetwork().cuda() # .to('cuda')
print(model2)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [ ]:
# 훈련되지 않은 모델로 검증하면 당연히 성능이 좋지 않다.
test_loop(test_dataloader, model2, loss_fn)

Test Error : 
 Accuracy : 9.2%, Avg Loss : 2.310001



In [ ]:
# 이전에 훈련된 가중치를 불러와서 model2에서 로드
model2.load_state_dict(torch.load('model_weights.pth'))
test_loop(test_dataloader, model2, loss_fn)

<ipython-input-16-ae9aae4d2e3d>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load('model_weights.pth'))


Test Error : 
 Accuracy : 85.2%, Avg Loss : 0.416231



# 훈련된 모델 자체를 저장 / 불러오기
모델의 구조를 모르는 경우 사용할 수 있는 대표적인 방법으로서 가중치만 저장한 경우보다 파일의 크기는 크지만 구조를 몰라도 모델을 사용할 수 있다는 장점이 있습니다.

In [ ]:
torch.save(model, 'model.pth')

In [ ]:
!ls -al | grep pth

-rw-r--r-- 1 root root 410496 Sep 24 01:35 model.pth
-rw-r--r-- 1 root root 409344 Sep 24 01:27 model_weights.pth


In [ ]:
model3 = torch.load('model.pth')
test_loop(test_dataloader, model3, loss_fn)

<ipython-input-21-557126119e97>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model3 = torch.load('model.pth')


Test Error : 
 Accuracy : 85.2%, Avg Loss : 0.416231

